In [59]:
import torch
import torch.nn as nn
import networkx as nx
import torch_geometric as pyg
from torch_geometric.datasets import KarateClub

In [60]:
from gcn_model import GCN

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
device

device(type='cuda')

In [63]:
dataset = KarateClub()

In [64]:
dataset.len()

1

In [65]:
graph = dataset[0].to(device)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
graph.train_mask

tensor([ True, False, False, False,  True, False, False, False,  True, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False,  True, False, False, False, False, False,
        False, False, False, False], device='cuda:0')

In [ ]:
graph.y

tensor([1, 1, 1, 1, 3, 3, 3, 1, 0, 1, 3, 1, 1, 1, 0, 0, 3, 1, 0, 1, 0, 1, 0, 0,
        2, 2, 0, 0, 2, 0, 0, 2, 0, 0], device='cuda:0')

In [ ]:
model = GCN([34, 16, 4]).to(device)
data = dataset[0].to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

In [ ]:
loss = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    l = loss(out[~data.train_mask], data.y[~data.train_mask])
    l.backward()
    optimizer.step()

In [ ]:
model.eval()

GCN(
  (convs): ModuleList(
    (0): GCNConv()
    (1): GCNConv()
  )
)

In [ ]:
preds = model(data.x, data.edge_index)

In [ ]:
preds[graph.train_mask].argmax(dim=1)

tensor([1, 0, 0, 0], device='cuda:0')

In [ ]:
preds[~graph.train_mask].argmax(dim=1)

tensor([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0], device='cuda:0')

In [ ]:
graph.y[~graph.train_mask]

tensor([1, 1, 1, 3, 3, 1, 1, 3, 1, 1, 1, 0, 0, 3, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0,
        2, 0, 0, 2, 0, 0], device='cuda:0')

In [ ]:
graph.y[graph.train_mask]

tensor([1, 3, 0, 2], device='cuda:0')

In [ ]:
~graph.train_mask

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.